In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
str(20201023)+"_"+str(1321)

'20201023_1321'

In [5]:
import glob
import os
import numpy as np
import torch
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision.transforms import v2
import torchvision.models as models
import torch.nn as nn
import cv2
from torch.utils.data import Dataset
from pathlib import Path
from typing import List, Tuple
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

date=20231228
time=2105
filedate=str(date)+"_"+str(time)
print(filedate)

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')

Channels=3
IMG_SIZE=224
epochlist=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]

Classes = ["higainashi", "houdo", "kanbotsu", "rokatahoukai"]
ClassNum = len(Classes)

testpath=r"C:\Users\AKIZUKI\JupyterProjects\QGISedited\test"
savepath=r"C:\Users\AKIZUKI\JupyterProjects\AerialPhoto_savedmodel_NOROAD"

'''
PytorchではDataloaderという,膨大なデータセットからでもメモリを圧迫せずに取り出せてforループにも対応するための枠組みがある
データセットをDataloaderが引っ張ってこれるような形式にするためにMyDataset(torch.utils.data.Dataset)というクラスを作れば，
あとはそのメソッドをtorch.utils.data.Datasetが勝手に使用してデータを加工してくれる
__init__, __getitem__, __len__をクラス内で必ず定義しなければならない
Dataloader内のデータはバッチごとにまとめられる
'''
class MyDataset(Dataset):
    def __init__(self, root: str, transforms, Classes) -> None:
        super().__init__()
        self.transforms = transforms
        self.Classes = Classes
        #globは複数のファイルのパスをまとめて取得する
        #訓練と訓練白黒の二個下のディレクトリから画像を取得
        self.data = list(sorted(Path(root).glob("*\*")))
        self.data2 = list(sorted(Path(root+"binary").glob("*\*")))



    # ここで取り出すデータを指定している
    def __getitem__(
            self,
            index: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        data = self.data[index]

        img1 = cv2.imread(str(data))
        img1 = cv2.resize(img1, (IMG_SIZE, IMG_SIZE))
        img1 = TF.to_tensor(img1)

        # データの変形 (transforms)
        transformed_img = self.transforms(img1)

        #ラベル貼り：dataというパスを/で区切ってリストにし，クラス名のところをラベルに格納
        #クラス名は文字列なので，self.Classesの要素と比較して一致するところの番号をラベルとする
        label = str(data).split("\\")[-2]
        label = torch.tensor(self.Classes.index(label))

        return transformed_img, label

    # この method がないと DataLoader を呼び出す際にエラーを吐かれる
    def __len__(self) -> int:
        return len(self.data)


#入力データに施す処理
transforms = v2.Compose([
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0,0,0], std=[0.2, 0.2, 0.2]),
])

testset= MyDataset(root=testpath, transforms=transforms, Classes=Classes)

testloader = DataLoader(dataset=testset,batch_size=len(testset),shuffle=True)

resnet50 = models.resnet50()

#modify first layer so it expects 4 input channels; all other parameters unchanged
resnet50.conv1 = torch.nn.Conv2d(Channels,64,kernel_size = (7,7),stride = (2,2), padding = (3,3), bias = False)
num_ftrs = resnet50.fc.in_features
#modifying final layer
resnet50.fc = nn.Linear(num_ftrs,ClassNum)

#lossfunction&optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)

def evaluate(testloader, model, loss_fn, optimizer):
    size_test = len(testloader.dataset)
    test_loss, test_correct = 0, 0
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    model.eval()
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in testloader:
            X=X.to(device)
            y=y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_correct /= size_test

    print(f'TestLoss: {test_loss:.4f} TestAcc: {test_correct:.4f}')

    #ndarrayにするため、ラベルyと推測結果predをgpuからcpuへ返す
    y=y.to(cpu)
    pred=pred.to(cpu)
   
    y=np.array(y)

    #predは各クラスの確率になってる（onehotに近い）ので実際のクラス番号に戻す
    pred_class=pred.argmax(1)
    pred_class=np.array(pred_class)
    
    
    #テストデータの混同行列を計算し可視化
    #scikitlearnの混同行列はラベルをonehotではなく実際のクラス番号にする必要がある
    #混同行列の見方は行が正解ラベルのクラス列が推定クラス
    print(confusion_matrix(y, pred_class))
    print(" ")

for e in epochlist:
    #モデル構築
    modelpath = Path(savepath+"\\"+str(e)+"\model_weights"+filedate+".pth")
    epochmodel = resnet50
    epochmodel.load_state_dict(torch.load(modelpath))
    #GPUにニューラルネットワークを渡す
    epochmodel=epochmodel.to(device)

    print("Model in Epoch", e)
    #テストデータで評価
    evaluate(testloader, epochmodel, loss_fn, optimizer)

print('Testing Complete!!!')

20231228_2105
Model in Epoch 10
TestLoss: 1.3095 TestAcc: 0.4615
[[16 13  0  3]
 [ 4 25  3  1]
 [ 3  9  2  3]
 [ 1 14  2  5]]
 
Model in Epoch 20
TestLoss: 1.5102 TestAcc: 0.4615
[[18  8  0  6]
 [ 4 21  3  5]
 [ 3  5  4  5]
 [ 1 11  5  5]]
 
Model in Epoch 30
TestLoss: 1.7363 TestAcc: 0.3750
[[ 9  9  0 14]
 [ 0 21  3  9]
 [ 0  6  3  8]
 [ 0 11  5  6]]
 
Model in Epoch 40
TestLoss: 1.8241 TestAcc: 0.4712
[[18  6  0  8]
 [ 3 25  2  3]
 [ 3  9  0  5]
 [ 0 12  4  6]]
 
Model in Epoch 50
TestLoss: 1.6921 TestAcc: 0.4904
[[22  7  1  2]
 [ 6 20  5  2]
 [ 3  6  4  4]
 [ 2 12  3  5]]
 
Model in Epoch 60
TestLoss: 1.9435 TestAcc: 0.4423
[[15  8  2  7]
 [ 3 21  7  2]
 [ 3  6  2  6]
 [ 0  7  7  8]]
 
Model in Epoch 70
TestLoss: 1.8660 TestAcc: 0.4808
[[21  7  0  4]
 [ 5 21  1  6]
 [ 4  7  3  3]
 [ 0 15  2  5]]
 
Model in Epoch 80
TestLoss: 2.1201 TestAcc: 0.5096
[[20  6  2  4]
 [ 3 26  2  2]
 [ 2 10  1  4]
 [ 1 11  4  6]]
 
Model in Epoch 90
TestLoss: 1.7347 TestAcc: 0.5096
[[21  6  1  4]
 [ 3 24 